In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from ucimlrepo import fetch_ucirepo

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#1. Wczytanie i filtrowanie danych
polish_companies_bankruptcy = fetch_ucirepo(id=365)

X = polish_companies_bankruptcy.data.features
y = polish_companies_bankruptcy.data.targets
y = y['class']
data = pd.concat([X, y.rename('Bankruptcy')], axis=1)

# Filtrowanie dla konkretnego roku
selected_year = 3
data = data[data['year'] == selected_year]
data = data.drop(columns=['year'])

In [ ]:
print(data)

In [ ]:
 # 2. Podstawowa eksploracja danych

print("\nPodstawowe informacje o danych:")
data.info()

print("\nPodstawowe statystyki opisowe:")
print(data.describe())

print("\nRozkład klas w kolumnie 'Bankruptcy':")
print(data['Bankruptcy'].value_counts())

In [ ]:
# 3. Obsługa braków danych

# Sprawdzenie braków w 'Bankruptcy'
print(f"Liczba braków w 'Bankruptcy' przed usunięciem: {data['Bankruptcy'].isnull().sum()}")
data = data.dropna(subset=['Bankruptcy'])
print(f"Liczba braków w 'Bankruptcy' po usunięciu: {data['Bankruptcy'].isnull().sum()}")

data.reset_index(drop=True, inplace=True)

# Sprawdzenie braków przed usuwaniem kolumn
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100
print("\nProcent braków w danych (przed usuwaniem kolumn):")
print(missing_percentage[missing_percentage > 0])

# Usuwanie kolumn z dużą liczbą braków (>40%)
threshold = 40
cols_to_drop = missing_percentage[missing_percentage > threshold].index
print(f"\nKolumny do usunięcia z powodu dużej liczby braków: {cols_to_drop}")
data = data.drop(columns=cols_to_drop)

# Sprawdzenie braków po usunięciu kolumn
missing_values = data.isnull().sum()
print("\nLiczba braków po usunięciu kolumn:")
print(missing_values[missing_values > 0])

# Wypełnianie braków medianą w pozostałych kolumnach
for col in data.columns:
    if data[col].isnull().sum() > 0:
        data[col].fillna(data[col].median(), inplace=True)

# Weryfikacja braków po przetwarzaniu
print("\nLiczba braków danych po przetwarzaniu:", data.isnull().sum().sum())  # Powinno być 0
data.reset_index(drop=True, inplace=True)

In [ ]:
#4. Analiza rozkładu klasy

class_distribution = data['Bankruptcy'].value_counts()
print("\nRozkład klas w zbiorze danych:")
print(class_distribution)

# Próba jest niezbilansowana
plt.figure(figsize=(6, 4))
sns.barplot(x=class_distribution.index, y=class_distribution.values, palette="viridis")
plt.title(f'Rozkład klas: Bankructwo (1) i Brak bankructwa (0) (rok {selected_year})')
plt.xlabel('Klasa')
plt.ylabel('Liczba próbek')
plt.show()

In [ ]:
#5. Skalowanie cech

print("Braki danych przed skalowaniem:")
print(features.isnull().sum().sum())  # Powinno być 0
scaler = MinMaxScaler()
features = data.drop(columns=['Bankruptcy'])
scaled_features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

# Dodanie klasy 'Bankruptcy' z powrotem
data = pd.concat([scaled_features, data['Bankruptcy']], axis=1)
print(data)

In [ ]:
# 5. Korelacja między cechami a klasą

correlations = data.corr()['Bankruptcy'].sort_values(ascending=False)
print("\nNajsilniejsze korelacje cech z klasą 'Bankruptcy':")
print(correlations.head(10))

# Wizualizacja macierzy korelacji
plt.figure(figsize=(12, 8))
sns.heatmap(data.corr(), annot=False, cmap='coolwarm', cbar=True)
plt.title("Macierz korelacji")
plt.show()

In [ ]:
# 6. Mutual Information jako alternatywa dla korelacji liniowej

X = data.drop(columns=['Bankruptcy'])
y = data['Bankruptcy']
mutual_info = mutual_info_classif(X, y, random_state=42)
mutual_info_series = pd.Series(mutual_info, index=X.columns).sort_values(ascending=False)
print("\nCechy o najwyższej informacji wzajemnej względem klasy 'Bankruptcy':")
print(mutual_info_series.head(10))

In [ ]:
#7. Podział na zbiór treningowy i testowy

X = data.drop(columns=['Bankruptcy'])  # Cechy
y = data['Bankruptcy']  # Klasa docelowa

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
#8. Wyrównanie klas za pomocą SMOTE na zbiorze treningowym

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print("\nRozkład klas po zastosowaniu SMOTE:")
print(y_train_resampled.value_counts())

# Weryfikacja liczby próbek
print(f"Liczba próbek w zbiorze treningowym po SMOTE: {X_train_resampled.shape[0]}")